In [10]:
import pandas as pd
import numpy as np
import os
import torch
from transformers import BertTokenizer
import torch.nn as nn
from transformers import BertModel
from torch.optim import Adam
from tqdm import tqdm

data_path = f'{os.path.dirname(os.path.dirname(os.getcwd()))}/dataset/'
data = pd.read_parquet(f'{data_path}/dataset_clean.parquet')
data = data.sample(frac=0.1)
#splitting dataset into train set (80%), validation set (10%) and test set (10%)
data_train, data_val, data_test = np.split(data.sample(frac=1, random_state=42), [int(.8*len(data)), int(.9*len(data))])

print(len(data_train),len(data_val), len(data_test))

55764 6970 6971


In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

labels = {'positive':0,
          'negative':1,
          'uncertainty':2,
          'litigious':3,
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, data):
        self.labels = np.array([labels[label] for label in data['Label']])
        self.texts = [tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for text in data['Text']]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

In [12]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(self.bert.config.hidden_size, 4)
        self.relu = nn.ReLU()

    def forward(self, input_id, attention_mask):
        _, output = self.bert(input_ids= input_id, attention_mask=attention_mask)
        output = self.dropout(output)
        output = self.linear(output)
        output = self.relu(output)

        return output

In [13]:


def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

In [14]:
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
train(model, data_train, data_val, LR, EPOCHS)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/27882 [00:02<?, ?it/s]


TypeError: dropout(): argument 'input' (position 1) must be Tensor, not str